In [1]:
#!./env/bin/ python3

# Import modules
import datetime as dt
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt

#import investingHelpers
#from modules.investingHelpers import Stock, Portfolio
import modules.investingHelpers as investment

# Helpful
# https://www.youtube.com/watch?v=t_vZDyQDUkk

In [14]:
class Joe():
    def __init__(self):
        self.num = 5
    
    def someFunc(self, otherNum):
        print(self.num + otherNum)
        return self.num + otherNum
    
    def outsideFunc(self, funcToCall):
        thefunction = getattr(self, funcToCall)
        thefunction(2)

In [15]:
guy = Joe()
guy.outsideFunc('someFunc')

7


In [5]:
myList = []

myList.append(investment.Stock('AAPL'))
myList.append(investment.Stock('MSFT'))
myList.append(investment.Stock('GOOGL'))

print(myList)
print([str(name) for name in myList])

# Get the stock we want to remove
stock = next((stock for stock in myList if stock.ticker == 'MSFT'), None)
if stock == None:
    raise Exception('Error: A stock with the ticker {} was not found in the stock list of this portfolio'.format('MSFT'))

myList.remove(stock)

print(myList)
print([str(name) for name in myList])

[<modules.investingHelpers.Stock object at 0x7f8895000130>, <modules.investingHelpers.Stock object at 0x7f8856e34c20>, <modules.investingHelpers.Stock object at 0x7f8856e78e00>]
['AAPL', 'MSFT', 'GOOGL']
[<modules.investingHelpers.Stock object at 0x7f8895000130>, <modules.investingHelpers.Stock object at 0x7f8856e78e00>]
['AAPL', 'GOOGL']


In [2]:
ts = investment.TransactionReader('./data/Test_History.tsv')

print(ts.tickers)

['Cash' 'ZWC.TO' 'VDY.TO' 'Cash' 'Cash' 'Cash' 'Cash' 'Cash' 'AAPL' 'REVS'
 'Cash' 'Cash' 'LMT' 'Cash' 'POW.TO' 'TCEHY' 'Cash' 'NCLH' 'NCLH' 'Cash'
 'ENB.TO' 'T.TO' 'CU.TO' 'Cash' 'Cash' 'Cash' 'Cash' 'MFC.TO' 'Cash'
 'Cash' 'Cash' 'MFC.TO' 'CU.TO' 'T.TO' 'BCE.TO' 'POW.TO' 'MFC.TO' 'ENB.TO'
 'CU.TO' 'TRP.TO' 'TD.TO' 'CNR.TO' 'MFC.TO' 'TRP.TO' 'Cash' 'Cash' 'REVS'
 'TCEHY' 'BTI' 'AQN.TO' 'ZWC.TO' 'BTI' 'CU.TO' 'AQN.TO' 'CU.TO' 'BTI'
 'BTI' 'CU.TO' 'CLX' 'CLX' 'Cash' 'AQN.TO' 'BCE.TO' 'SU.TO' 'VDY.TO'
 'SU.TO' 'BCE.TO' 'BCE.TO' 'BCE.TO' 'CU.TO' 'CU.TO' 'CU.TO' 'AAPL'
 'TRP.TO' 'BTI']


In [30]:
transactions = np.genfromtxt('./data/Test_History.tsv', delimiter='\t', skip_header=2, autostrip=True, dtype=str)

print(transactions.shape)

print(transactions[:,0][:2])
#d = '07/12/2021'
d = transactions[:,0][:2]
#actual_date = dt.datetime(d)
actual_date = np.array([dt.datetime.strptime(t, '%m/%d/%Y') for t in d])

#actual_date = dt.datetime.strptime(d, '%m/%d/%Y')
print(actual_date)
print(type(actual_date))

# %m %d %y

(75, 10)
['07/09/2021' '07/12/2021']
[datetime.datetime(2021, 7, 9, 0, 0) datetime.datetime(2021, 7, 12, 0, 0)]
<class 'numpy.ndarray'>


In [33]:
date_dict = dict.fromkeys(actual_date, [])

print(date_dict)

print(len(date_dict[actual_date[0]]))

{datetime.datetime(2021, 7, 9, 0, 0): [], datetime.datetime(2021, 7, 12, 0, 0): []}
0


In [2]:
end_date = dt.datetime.now()
start_date = dt.datetime(2023,2,4) # Year, month, day

investment.GetWeekdays(start_date, end_date)

array([datetime.datetime(2023, 2, 6, 0, 0),
       datetime.datetime(2023, 2, 7, 0, 0),
       datetime.datetime(2023, 2, 8, 0, 0),
       datetime.datetime(2023, 2, 9, 0, 0),
       datetime.datetime(2023, 2, 10, 0, 0),
       datetime.datetime(2023, 2, 13, 0, 0),
       datetime.datetime(2023, 2, 14, 0, 0),
       datetime.datetime(2023, 2, 15, 0, 0),
       datetime.datetime(2023, 2, 16, 0, 0),
       datetime.datetime(2023, 2, 17, 0, 0),
       datetime.datetime(2023, 2, 20, 0, 0)], dtype=object)

In [12]:
# Choose a date range and pull data
end_date = dt.datetime.now()
start_date = dt.datetime(2020,1,1) # Year, month, day

stock_list = ['VFV.TO']

yf.pdr_override()
df = pdr.get_data_yahoo(stock_list, start_date, end_date, progress=False)
print(type(df.iloc[:, [4]]))
#close = df.Close
#print(type(close))

#df.head()
#print(df.shape)
#df.head()

<class 'pandas.core.frame.DataFrame'>


In [2]:
end_date = dt.datetime.now()
start_date = dt.datetime(2020,1,1) # Year, month, day

myStock = Stock('Apple', 'AAPL')

myStock.PullData(start_date, end_date)

myStock.Buy(5)

print(myStock.units)
#myStock.close_prices
print(myStock)

5
Apple (AAPL)


In [15]:
# Try to access exchagne rate data

end_date = dt.datetime.now()
start_date = dt.datetime(2020,1,1)

myExchange = investment.Stock('USDCAD=X')

print(myExchange)

myExchange.PullData(start_date, end_date)

print(myExchange.close_prices)
print(type(myExchange.close_prices))
print(type(myExchange.close_prices['Adj Close'][0]))
print(myExchange.close_prices['Adj Close'][0])
print(myExchange.close_prices['Adj Close'].keys)
print(myExchange.close_prices['Adj Close'][dt.datetime(2023,2,16)])
#print(myExchange.close_prices[dt.datetime(2023,2,16)])
#print(myExchange.close_prices[str(dt.datetime(2023,2,16))])

USDCAD=X
            Adj Close
Date                 
2020-01-01    1.30020
2020-01-02    1.29730
2020-01-03    1.29830
2020-01-06    1.29866
2020-01-07    1.29639
...               ...
2023-02-13    1.33574
2023-02-14    1.33331
2023-02-15    1.33406
2023-02-16    1.33906
2023-02-17    1.34730

[818 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.float64'>
1.3001999855041504
<bound method Series.keys of Date
2020-01-01    1.30020
2020-01-02    1.29730
2020-01-03    1.29830
2020-01-06    1.29866
2020-01-07    1.29639
               ...   
2023-02-13    1.33574
2023-02-14    1.33331
2023-02-15    1.33406
2023-02-16    1.33906
2023-02-17    1.34730
Name: Adj Close, Length: 818, dtype: float64>
1.3390599489212036


In [19]:
msft = yf.Ticker('VFV.TO')
name = msft.info['longName']
print(name)

Vanguard S&P 500 Index ETF


In [4]:
# Choose a date range and pull data
end_date = dt.datetime.now()
start_date = dt.datetime(2020,1,1) # Year, month, day

myPortfolio = Portfolio()

myPortfolio.AddStock('MSFT')
myPortfolio.AddStock('AAPL')
myPortfolio.AddStock('VFV.TO')

for stock in myPortfolio.stock_list:
    stock.PullData(start_date, end_date)

print([str(stock) for stock in myPortfolio.stock_list])

#stock = next((stock for stock in myPortfolio.stock_list if stock.ticker == 'Joe'), None)
print(stock)

['MSFT', 'AAPL', 'VFV.TO']
None


In [14]:
a = []
#a.append(np.array(['key1', 5]))
#a.append(np.array(['key2', 1]))

a.append(['key1', 5])
a.append(['key2', 1])

a = np.array(a)
print(a[0])
#print(a.shape())
print(a[:,0])

['key1' '5']
['key1' 'key2']


In [9]:
b = {'CAD': 1, 'USD': 2}

print(b.keys())

dict_keys(['CAD', 'USD'])


In [12]:
a = 0.1223
a -= 1
print(a)

-0.8777
